Load libraries

In [47]:
require(ggplot2)
require(reshape2)
require(lme4)
require(compiler)
require(parallel)
require(car)
require(boot)
require(dplyr)

# Load data and set factors

In [48]:
mydata <- read.csv("C:\\Users\\Sarah\\Documents\\Personal Content\\Lab_study_data\\all_massaged_data\\dataframe_all_factors_for_analysis.txt",sep = '\t')
# # SID is the student number
mydata$sid <- factor(mydata$sid)
mydata$sim_index <- factor(mydata$sim_index)
mydata$lab_experience <- factor(mydata$lab_experience)
mydata$similar_sim <- factor(mydata$similar_sim)
mydata$CVS_graph <- factor(mydata$cvs_graph)
mydata$CVS_table <- factor(mydata$cvs_table)
# mydata$pre <- factor(mydata$pre)
# mydata$main <- factor(mydata$main)

In [49]:
head(mydata)
colnames(mydata)

sid,sim,variable,pre,main,cvs_graph,cvs_table,qual_score,quant_score,activity_order,...,CVS_context,use_table,use_graph,use_concentration,use_width,use_area,use_separation,use_all_vars,CVS_graph,CVS_table
10127163,L,Concentration,0,2,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10127163,L,Width,0,2,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10127163,C,Area,2,2,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10127163,C,Separation,2,2,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10232160,L,Concentration,0,0,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10232160,L,Width,0,0,0,0,1,1,LC,...,0,1,1,1,1,1,1,4,0,0


[1] "sid"                       "sim"                      
 [3] "variable"                  "pre"                      
 [5] "main"                      "cvs_graph"                
 [7] "cvs_table"                 "qual_score"               
 [9] "quant_score"               "activity_order"           
[11] "sim_index"                 "similar_sim"              
[13] "lab_experience"            "prior_number_virtual_labs"
[15] "overall.POCC"              "PC1"                      
[17] "PC2"                       "pre_with_ident"           
[19] "main_with_ident"           "CVS_context"              
[21] "use_table"                 "use_graph"                
[23] "use_concentration"         "use_width"                
[25] "use_area"                  "use_separation"           
[27] "use_all_vars"              "CVS_graph"                
[29] "CVS_table"

# Stat model 1: Predicting main model scores

We try to predict the type of model in the main worksheet (0,1 or 2 for neither, qual or quant). To do so, we have 10 independant variables:
* sim_index (1 or 2, wither it was student's 1st or 2nd activity)
* variable (thus don't include sim)
* CVS_graph (0 or 1)
* CVS_table (0 or 1)
* pre (0,1,2)
* student attributes: lab_experience, similar_sim, prior_number_virtual_labs, attitude components

We do so using a repeated measures anova where we have repeat measures of "variable" by student.

**Should we remove students with max pre for a certain variables???**

## Using type 1 and aov package

In [50]:
am1 <- aov(main
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + Error(sid/variable), data=mydata)
summary(am1)


Error: sid
                           Df Sum Sq Mean Sq F value   Pr(>F)    
cvs_table                   1  21.10  21.101  31.621 1.01e-07 ***
cvs_graph                   1   2.83   2.825   4.234   0.0415 *  
pre                         1  12.57  12.571  18.839 2.74e-05 ***
lab_experience              1   0.23   0.233   0.350   0.5551    
similar_sim                 1   0.30   0.302   0.452   0.5024    
prior_number_virtual_labs   1   0.20   0.204   0.306   0.5809    
overall.POCC                1   0.34   0.342   0.513   0.4751    
PC1                         1   0.02   0.019   0.029   0.8658    
PC2                         1   0.58   0.584   0.874   0.3514    
Residuals                 137  91.42   0.667                     
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Error: sid:variable
             Df Sum Sq Mean Sq F value   Pr(>F)    
cvs_table     1   2.84   2.841  12.470 0.000458 ***
cvs_graph     1   4.23   4.231  18.573 2.03e-05 ***
variable      3   

We see that the variance from having repeated measures for each student (first table) has been removed from the overall model error and thus separated in the output.

Looking the second table we notice that both cvs are significant, cvs_graph more significant than cvs_table, variable is significant and so is sim_index.

eta squares, and post hoc test are coming!

## Using type 3 and anova package

We create a dataframe to tell the anova how "variable" leades to multiple between subject measures.

For info on why Type 1,2,3: https://mcfromnz.wordpress.com/2011/03/02/anova-type-iiiiii-ss-explained/

In [69]:
variables <- factor(cbind("Width","Area","Concentration","Separation"))
measures <- factor(rep(c("pre", "main", "cvs_table","cvs_graph","sim_index","similar_sim"), c(4,4,4,4,4,4)),
levels=c("pre", "main", "cvs_table","cvs_graph","sim_index","similar_sim"))
vars <- ordered(rep(variables, 6))
idata <- data.frame(measures,vars)
idata

measures,vars
pre,Width
pre,Area
pre,Concentration
pre,Separation
main,Width
main,Area
main,Concentration
main,Separation
cvs_table,Width
cvs_table,Area


In [70]:
mod.ok <- lm(main
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2, data=mydata)
av.ok <- Anova(mod.ok, idata=idata, type=3, idesign=~pre*variable*main*cvs_table*cvs_graph*sim_index*similar_sim)
av.ok

,Sum Sq,Df,F value,Pr(>F)
(Intercept),5.5733105,1,16.2300511,6.364673e-05
cvs_table,0.7448854,1,2.1691824,1.413492e-01
cvs_graph,6.4379911,1,18.7480893,1.760392e-05
variable,2.5465220,3,2.4719110,6.086836e-02
sim_index,2.2707103,1,6.6125410,1.037689e-02
pre,6.1760252,1,17.9852179,2.594679e-05
lab_experience,0.1486979,1,0.4330236,5.107728e-01
similar_sim,0.1663807,1,0.4845176,4.866649e-01
prior_number_virtual_labs,0.1983452,1,0.5776016,4.475660e-01
overall.POCC,0.7600863,1,2.2134491,1.373616e-01


In [68]:
mod.ok <- lm(main
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2, data=mydata)
variables <- factor(cbind("Width","Area","Concentration","Separation"))
idata <- data.frame(variables)
av.ok <- Anova(mod.ok, idata=idata, idesign=~variable,type=3)
av.ok

,Sum Sq,Df,F value,Pr(>F)
(Intercept),5.5733105,1,16.2300511,6.364673e-05
cvs_table,0.7448854,1,2.1691824,1.413492e-01
cvs_graph,6.4379911,1,18.7480893,1.760392e-05
variable,2.5465220,3,2.4719110,6.086836e-02
sim_index,2.2707103,1,6.6125410,1.037689e-02
pre,6.1760252,1,17.9852179,2.594679e-05
lab_experience,0.1486979,1,0.4330236,5.107728e-01
similar_sim,0.1663807,1,0.4845176,4.866649e-01
prior_number_virtual_labs,0.1983452,1,0.5776016,4.475660e-01
overall.POCC,0.7600863,1,2.2134491,1.373616e-01


## Repeat to compare cvs_table to cvs_graph
We rerun the model for all students who did cvs_table to see if doing cvs_graph actually helps more.

In [53]:
# am1_graph <- aov(main
#     ~ cvs_graph + variable + sim_index + pre
#     + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
#     +  PC1 + PC2 + Error(sid/variable), data=filter(mydata, cvs_table == 1))
# summary(am1_graph)

# Stat model 2: Predicting transfer data

In [55]:
am2 <- aov(quant_score
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + Error(sid/variable), data=mydata)
summary(am2)


Error: sid
                           Df Sum Sq Mean Sq F value Pr(>F)  
cvs_table                   1   0.05  0.0542   0.130 0.7195  
cvs_graph                   1   2.04  2.0383   4.875 0.0289 *
pre                         1   1.58  1.5804   3.780 0.0539 .
lab_experience              1   0.06  0.0646   0.154 0.6949  
similar_sim                 1   0.62  0.6225   1.489 0.2245  
prior_number_virtual_labs   1   2.11  2.1087   5.044 0.0263 *
overall.POCC                1   0.19  0.1877   0.449 0.5040  
PC1                         1   0.05  0.0549   0.131 0.7177  
PC2                         1   0.43  0.4314   1.032 0.3115  
Residuals                 137  57.28  0.4181                 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Error: sid:variable
             Df Sum Sq Mean Sq F value Pr(>F)  
cvs_table     1   0.06  0.0557   0.410 0.5222  
cvs_graph     1   0.24  0.2397   1.766 0.1845  
variable      3   1.35  0.4512   3.325 0.0197 *
sim_index     1   0.24  0.

As expected, CVS doesn't predict quant transfer scores.

Post hoc by variable and similar sim (weither students had worked with a similar sim before) coming soon.

## Using type 3 and anova package

In [58]:
mod.ok <- lm(quant_score
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2, data=mydata)
variables <- factor(cbind("Width","Area","Concentration","Separation"))
idata <- data.frame(variables)
av.ok <- Anova(mod.ok, idata=idata, idesign=~variables,type=3)
av.ok

,Sum Sq,Df,F value,Pr(>F)
(Intercept),3.769096e+00,1,18.00758652,2.565288e-05
cvs_table,2.625584e-01,1,1.25442366,2.631777e-01
cvs_graph,2.584682e-01,1,1.23488160,2.669250e-01
variable,1.541581e+00,3,2.45506674,6.223762e-02
sim_index,7.500589e-02,1,0.35835513,5.496573e-01
pre,1.438331e+00,1,6.87190262,8.988296e-03
lab_experience,7.416440e-02,1,0.35433477,5.519041e-01
similar_sim,2.510144e-02,1,0.11992697,7.292406e-01
prior_number_virtual_labs,1.875921e+00,1,8.96257298,2.874305e-03
overall.POCC,1.370463e-02,1,0.06547651,7.981322e-01


# Stat model 3: Predicting the use of CVS

## For cvs_table

In [ ]:
# mydata$variable <- relevel(mydata$variable, "Width")
mixed <- glmer(
    cvs_table
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

## For cvs_graph

In [ ]:
mixed <- glmer(
    cvs_graph
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

# Example from R package "car"
See page 9 of https://cran.r-project.org/web/packages/car/car.pdf

In [13]:
?OBrienKaiser

In [15]:
OBrienKaiser

treatment,gender,pre.1,pre.2,pre.3,pre.4,pre.5,post.1,post.2,post.3,post.4,post.5,fup.1,fup.2,fup.3,fup.4,fup.5
control,M,1,2,4,2,1,3,2,5,3,2,2,3,2,4,4
control,M,4,4,5,3,4,2,2,3,5,3,4,5,6,4,1
control,M,5,6,5,7,7,4,5,7,5,4,7,6,9,7,6
control,F,5,4,7,5,4,2,2,3,5,3,4,4,5,3,4
control,F,3,4,6,4,3,6,7,8,6,3,4,3,6,4,3
A,M,7,8,7,9,9,9,9,10,8,9,9,10,11,9,6
A,M,5,5,6,4,5,7,7,8,10,8,8,9,11,9,8
A,F,2,3,5,3,2,2,4,8,6,5,6,6,7,5,6
A,F,3,3,4,6,4,4,5,6,4,1,5,4,7,5,4
B,M,4,4,5,3,4,6,7,6,8,8,8,8,9,7,8


In [20]:
phase <- factor(rep(c("pretest", "posttest", "followup"), c(5, 5, 5)),
levels=c("pretest", "posttest", "followup"))
hour <- ordered(rep(1:5, 3))
idata <- data.frame(phase, hour)
idata

phase,hour
pretest,1
pretest,2
pretest,3
pretest,4
pretest,5
posttest,1
posttest,2
posttest,3
posttest,4
posttest,5


In [31]:
mod.ok <- lm(cbind(pre.1, pre.2, pre.3, pre.4, pre.5,
post.1, post.2, post.3, post.4, post.5,
fup.1, fup.2, fup.3, fup.4, fup.5) ~ treatment*gender,
data=OBrienKaiser)
(av.ok <- Anova(mod.ok, idata=idata, idesign=~phase*hour,type=3))
summary(av.ok, multivariate=FALSE)


Type III Repeated Measures MANOVA Tests: Pillai test statistic
                            Df test stat approx F num Df den Df    Pr(>F)    
(Intercept)                  1   0.96736  296.389      1     10 9.241e-09 ***
treatment                    2   0.44075    3.940      2     10 0.0547069 .  
gender                       1   0.26789    3.659      1     10 0.0848003 .  
treatment:gender             2   0.36350    2.855      2     10 0.1044692    
phase                        1   0.81363   19.645      2      9 0.0005208 ***
treatment:phase              2   0.69621    2.670      4     20 0.0621085 .  
gender:phase                 1   0.06614    0.319      2      9 0.7349696    
treatment:gender:phase       2   0.31060    0.919      4     20 0.4721498    
hour                         1   0.93286   24.315      4      7 0.0003345 ***
treatment:hour               2   0.31634    0.376      8     16 0.9183275    
gender:hour                  1   0.33922    0.898      4      7 0.5129764    



Univariate Type III Repeated-Measures ANOVA Assuming Sphericity

                                SS num Df Error SS den Df        F    Pr(>F)
(Intercept)                 6759.3      1  228.056     10 296.3888 9.241e-09
treatment                    179.7      2  228.056     10   3.9405  0.054707
gender                        83.4      1  228.056     10   3.6591  0.084800
treatment:gender             130.2      2  228.056     10   2.8555  0.104469
phase                        129.5      2   80.278     20  16.1329 6.732e-05
treatment:phase               77.9      4   80.278     20   4.8510  0.006723
gender:phase                   2.3      2   80.278     20   0.2828  0.756647
treatment:gender:phase        10.2      4   80.278     20   0.6366  0.642369
hour                         104.3      4   62.500     40  16.6857 4.027e-08
treatment:hour                 1.2      8   62.500     40   0.0933  0.999245
gender:hour                    2.8      4   62.500     40   0.4503  0.771559
treatment: